In [1]:
from main import main
import argparse

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', help='Path to the config file', default='./default_config.yml')
parser.add_argument('--log-dir', help='Path to the log directory', default='../log/')
parser.add_argument('--posters', help='Path to the posters', default='../data/posters/')
parser.add_argument('--models-dir', help='Path to the saved models', default='../data/models/')
parser.add_argument('--preproc', help='Path to the preprocessed data', default='../data/preprocessed/')
parser.add_argument('--split', help='Path to the splitted data', default='../data/splitted/')
parser.add_argument('--database', help='Path to the databse csv', default='../data/poster_data.csv')
parser.add_argument('--csv', help='Path to the clean csv', default='../data/')
parser.add_argument('-s', '--save', help='Save model', action='store_true')
parser.add_argument('-v', '--verbose', help='Verbose', action='store_true')

str_args = '-v'.split()
args, _ = parser.parse_known_args(str_args)

In [8]:
model, test_posters, test_genres, test_ids, movies, predicted_genres = main(args)

Database already cleaned


 10%|▉         | 715/7196 [00:00<00:00, 7141.70it/s]

Posters database downloading


100%|██████████| 7196/7196 [00:01<00:00, 6678.89it/s]


Database downloaded
Data already preprocessed
Data already splitted
Model already trained
0.08176470588235295


In [16]:
import yaml

config = yaml.safe_load(open(args.config))
genres_inv = {config['genres'][k]: k for k in config['genres'].keys()}
print(genres_inv)

{0: 'Action', 1: 'Animation', 2: 'Aventure', 3: 'Biopic', 4: 'Comédie', 5: 'Comédie-dramatique', 6: 'Comédie-musicale', 7: 'Documentaire', 8: 'Drame', 9: 'Epouvante-horreur', 10: 'Fantastique', 11: 'Historique', 12: 'Policier', 13: 'Romance', 14: 'Science-fiction', 15: 'Thriller', 16: 'Western'}


In [18]:
import numpy as np

predictions = [] 
for pred in predicted_genres:
    first3_index = np.argsort(pred)[-3:]
    predictions.append([genres_inv[k] for k in first3_index])
        
ground_truth = []
for truth in test_genres:
    first3_index = np.nonzero(truth)[0]
    ground_truth.append([genres_inv[k] for k in first3_index])

In [22]:
r = np.random.permutation(100)[:10]
for i in r:
    print('{}: truth: {}, pred: {}'.format(movies.loc[test_ids[i], 'title'], ground_truth[i], predictions[i]))
#     print('Ground truth: {}, real: {}'.format(ground_truth[i], movies.loc[test_ids[i], 'genres']))

L'Insoumis: truth: ['Drame'], pred: ['Thriller', 'Comédie', 'Drame']
L'Echange des Princesses: truth: ['Drame', 'Historique'], pred: ['Thriller', 'Comédie', 'Drame']
Le Graphique de Boscop: truth: ['Comédie'], pred: ['Thriller', 'Comédie', 'Drame']
Absent: truth: ['Drame'], pred: ['Thriller', 'Comédie', 'Drame']
Petite amie: truth: ['Comédie-dramatique'], pred: ['Thriller', 'Drame', 'Comédie']
The Whole Gritty City: truth: ['Documentaire'], pred: ['Thriller', 'Comédie', 'Drame']
Super 8: truth: ['Science-fiction'], pred: ['Comédie', 'Thriller', 'Drame']
La Terre éphémère: truth: ['Drame'], pred: ['Thriller', 'Comédie', 'Drame']
Dieu n'est pas mort: truth: ['Drame'], pred: ['Thriller', 'Comédie', 'Drame']
Comment tuer son boss 2: truth: ['Comédie'], pred: ['Thriller', 'Comédie', 'Drame']
